# Evidence 1. Integrative Activity

Team Members:
- Diego Aguilar Torres - A01657884
- Katia Geraldine Vidals Estrever - A01657587
- David Alonso Chang Ortega - A01658631
- Dana Mendoza Palacios Roji - A01658253



## Pip Install

In [ ]:
pip install seaborn

In [111]:
pip install --upgrade mesa[rec]

## Import Dependencies

In [ ]:
pip install mesa

In [ ]:
print(mesa.__version__)

NameError: name 'mesa' is not defined

In [ ]:
pip install --upgrade mesa

In [ ]:
print(mesa.__version__)

1.0.0


In [112]:
import mesa
print(mesa.__version__)

# Data visualization tools.
import seaborn as sns

# Has multi-dimensional arrays and matrices. Has a large collection of
# mathematical functions to operate on these arrays.
import numpy as np

# Data manipulation and analysis.
import pandas as pd

3.0.3


In [ ]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
# from mesa.visualization import CanvasGrid
# from mesa.visualization.ModularVisualization import ModularServer
import random
import matplotlib.pyplot as plt

##Car Agent

In [ ]:
!pip install mesa==3.0.0

In [113]:
class Car(mesa.Agent):
    def __init__(self, unique_id, start_parking, target_parking, model):
        super().__init__(model)
        self.unique_id = unique_id
        self.start_parking = start_parking
        self.target_parking = target_parking
        self.state = "idle"
        self.direction = None
        self.value = self.model.grid.properties["city_objects"].data[start_parking]     # Valor de estacionamiento
        self.directions_dict_not = None
        self.last_pos = None


        y_change_up = [6, 7, 14, 15, 18, 19, 22, 23]
        y_change_down = [0, 1, 6, 7, 12, 13]
        x_change_right = [14, 15, 18, 19, 22, 23]
        x_change_left = [0, 1, 5, 6, 7, 12, 13, 18, 19]


        self.directions_dict_not = {
          "not_up": y_change_down,
          "not_down": y_change_up,
          "not_right": x_change_left,
          "not_left": x_change_right,
        }


    def step(self):
      if self.pos == self.start_parking:
        self.move()
      else:
        if self.pos != self.target_parking:
          self.move()
        else:
          self.state = "idle"
          self.direction = None

    def move(self):
      print(self.pos, self.direction)
      # Get the neighborhood of the current position, excluding the center
      possible_steps = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)

      # Filter positions to only include cells with a value of zero
      valid_steps = []

      for step in possible_steps:
          if self.model.grid.properties["city_objects"].data[step] == 0:
              if (self.last_pos and step == self.last_pos):
                  continue
              valid_steps.append(step)


      # Debug: Print the valid steps and their corresponding values in city_objects
      print("Current position:", self.pos)
      print("Valid steps and their values:")

      print("--> All valid steps: ")
      for step in valid_steps:
        value_step = self.model.grid.properties["city_objects"].data[step]
        print(f"Coordinate: {step}, Value: {value_step}")

      print("Check steps:")

      for step in valid_steps:
        value_step = self.model.grid.properties["city_objects"].data[step]
        current_x, current_y = self.pos  # Current position
        step_x, step_y = step            # Valid step coordinates
        print(f"Coordinate: {step}, Step_x: {step_x},  Step_y: {step_y}")

        if self.direction is not None:
          # Determine the axis of movement
          print("Direction: ", self.direction)
          print("Step_x", step_x, "Current_x", current_x)
          print("Step_y", step_y, "Current_y", current_y)
          if step_y != current_y:
            if current_y > step_y and step_x in self.directions_dict_not["not_left"]:
              print("deleteleft")
              valid_steps.remove(step)

            if current_y < step_y and step_x in self.directions_dict_not["not_right"]:
              print("deleteright")
              valid_steps.remove(step)

            if self.direction == "left" and current_y < step_y:
              print("deleterepeatright: ", step_y)
              valid_steps.remove(step)

            elif self.direction == "right" and current_y > step_y:
              print("deleterepeatleft: ", step_y)
              valid_steps.remove(step)

          elif step_x != current_x:
            if current_x > step_x and step_y in self.directions_dict_not["not_up"]:
              print("deleteup")
              valid_steps.remove(step)

            if current_x < step_x and step_y in self.directions_dict_not["not_down"]:
              print("deletedown")
              valid_steps.remove(step)

            if self.direction == "up" and current_x < step_x:
              print("deletedrepeatup: ", step_x)
              valid_steps.remove(step)

            elif self.direction == "down" and current_x > step_x:
              print("deleterepeatdown: ", step_x)
              valid_steps.remove(step)

      print("Old Position: ", self.pos)
      self.model.grid.properties["city_objects"].set_cell(self.pos, 0)  # Reset previous position
      self.last_pos = self.pos
      new_position = self.random.choice(valid_steps)
      if new_position[1] < self.pos[1]:  # y decreased
          self.direction = "left"
      elif new_position[1] > self.pos[1]:  # y increased
          self.direction = "right"
      elif new_position[0] < self.pos[0]:  # x decreased
          self.direction = "up"
      elif new_position[0] > self.pos[0]:  # x increased
          self.direction = "down"
      self.model.grid.move_agent(self, new_position)
      self.model.grid.properties["city_objects"].set_cell(self.pos, -1)
      self.state = "moving"
      print("New position:", self.pos)


## Multiagent System


In [114]:

class SemaphoreAgent(mesa.Agent):
    """An agent representing a traffic semaphore"""

    def __init__(self, unique_id, model, positions, green_duration=5, red_duration=5):
        super().__init__(model)
        self.positions = positions
        self.light_state = "green" if unique_id % 2 == 0 else "red"
        self.green_duration = green_duration
        self.red_duration = red_duration
        self.step_counter = 0


    def update_state(self):
        for position in self.positions:
            if self.light_state == "green":
                  state_value = 18
            else:
                  state_value = 19
            self.model.grid.properties["city_objects"].set_cell(position, state_value)


    def toggle_light(self):
        self.update_state()
        self.step_counter += 1
        if (self.light_state == "green" and self.step_counter >= self.green_duration) or (self.light_state == "red" and self.step_counter >= self.red_duration):
            if self.light_state == "red":
                self.light_state = "green"
            else:
                self.light_state = "red"
            self.step_counter = 0
            self.update_state()


class CityModel(mesa.Model):
    """A model of a city with some number of cars, semaphores, buildings, parking lots and a roundabout."""

    def __init__(self, cars, seed=None):
        super().__init__(seed=seed)
        self.num_cars = cars
        self.cars_list = []
        self.grid = mesa.space.MultiGrid(24, 24, False)
        self.initialize_city_objects()
        self.initialize_semaphores()
        self.steps = 0

    def initialize_cars(self): #agregadoKatia
        for i in range(self.num_cars):
          start_parking = self.parking_lots[i % len(self.parking_lots)]  # Desde el Primer estacionamiento
          target_parking = self.parking_lots[-(i % len(self.parking_lots) + 1)]  # Desde el Último estacionamiento

          car = Car(unique_id=-(i+1), start_parking=start_parking, target_parking=target_parking, model=self)
          self.cars_list.append(car)
          self.grid.place_agent(car, start_parking)

          # car = Car(-(i+1), self.parking_lots[0], self.parking_lots[self.parking_lots.count() - i], self)
          # self.cars.append(car)
          # self.grid.place_agent(car, self.parking_lots[0])

        parking_lots = list(self.parking_lot_map.keys())



    def initialize_semaphores(self):
        self.semaphores = {}
        semaphores_positions = {
            1: [(18, 2), (19, 2)],
            2: [(17, 0), (17, 1)],
            3: [(22, 5), (23, 5)],
            4: [(2, 6), (2, 7)],
            5: [(0, 8), (1, 8)],
            6: [(7, 6), (7, 7)],
            7: [(5, 8), (6, 8)],
            8: [(21, 6), (21, 7)],
            9: [(14, 17), (15, 17)],
            10: [(16, 18), (16, 19)]
        }

        for semaphore_id, positions in semaphores_positions.items():
            semaphore = SemaphoreAgent(unique_id=semaphore_id, model = self, positions=positions)
            self.semaphores[semaphore_id] = semaphore
            self.grid.place_agent(semaphore, positions[0])


    def initialize_city_objects(self):
        """Initialize buildings, parking lots and a roundabout on the grid using PropertyLayer."""
        city_objects_layer = mesa.space.PropertyLayer("city_objects", self.grid.width, self.grid.height, np.int64(0), np.int64)
        self.grid.properties["city_objects"] = city_objects_layer

        #Define buildings coordinates
        buildings = [(2, 2), (3, 2), (4, 2), (5, 2), (6, 2), (7, 2), (8, 2), (10, 2), (11, 2), (3, 3), (4, 3), (5, 3), (6, 3), (7, 3), (8, 3), (9, 3), (10, 3), (11, 3),
                     (2, 4), (3, 4), (4, 4), (5, 4), (6, 4), (7, 4), (8, 4), (9, 4), (10, 4), (2, 5), (3, 5), (4, 5), (5, 5), (7, 5), (8, 5), (9, 5), (10, 5), (11, 5),
                     (2, 8), (3, 8), (4, 8), (7, 8), (9, 8), (10, 8), (11, 8), (2, 9), (3, 9), (4, 9), (7, 9), (8, 9), (9, 9), (10, 9), (11, 9), (2, 10), (3, 10), (7, 10),
                     (8, 10), (9, 10), (10, 10), (2, 11), (3, 11), (4, 11), (7, 11), (8, 11), (9, 11), (10, 11), (11, 11), (16, 2), (17, 2), (20, 2), (21, 2), (16, 3), (20, 3),
                     (21, 3), (16, 4), (17, 4), (21, 4), (16, 5), (17, 5), (20, 5), (21, 5), (16, 8), (17, 8), (20, 8), (21, 8), (16, 9), (17, 9), (20, 9), (17, 10), (20, 10), (21, 10),
                     (16, 11), (17, 11), (20, 11), (21, 11), (2, 16), (3, 16), (4, 16), (5, 16), (8, 16), (9, 16), (10, 16), (11, 16), (3, 17), (4, 17), (5, 17), (8, 17), (9, 17), (10, 17),
                     (11, 17), (2, 18), (3, 18), (4, 18), (5, 18), (8, 18), (9, 18), (10, 18), (11, 18), (2, 19), (3, 19), (4, 19), (5, 19), (8, 19), (9, 19), (10, 19), (11, 19), (2, 20),
                     (3, 20), (4, 20), (9, 20), (10, 20), (11, 20), (2, 21), (3, 21), (4, 21), (5, 21), (8, 21), (9, 21), (10, 21), (11, 21), (16, 16), (17, 16), (18, 16), (19, 16), (20, 16),
                     (21, 16), (16, 17), (18, 17), (20, 17), (21, 17), (16, 20), (17, 20), (18, 20), (20, 20), (21, 20), (16, 21), (17, 21), (18, 21), (19, 21), (20, 21), (21, 21)]
        for (x, y) in buildings:
            position = (x, y)
            if self.grid.properties["city_objects"].data[x, y] == 0:
                self.grid.properties["city_objects"].set_cell(position, 20)

        #Define parking lots coordinates
        self.parking_lot_map = {}
        self.parking_lots = [(9, 2), (2, 3), (17, 3), (11, 4), (20, 4), (6, 5), (8, 8), (21, 9), (4, 10), (11, 10), (16, 10), (2, 17), (17, 17), (19, 17), (5, 20), (8, 20), (19, 20)]
        for parking_id, (x, y) in enumerate(self.parking_lots, start = 1):
            position = (x, y)
            value = self.grid.properties["city_objects"].data[position]
            if value == 0:
              self.grid.properties["city_objects"].set_cell(position, parking_id)
              self.parking_lot_map[parking_id] = position

        #Define cars coordinates
        self.initialize_cars()

        for car in self.cars_list:
          position = car.pos
          value = self.grid.properties["city_objects"].data[position]
          #print(f"value {value}")  #Valor de estacionamiento
          if value is not None:
            self.grid.properties["city_objects"].set_cell(position, -1)


        #Define roundabout coordinates
        roundabout = [(13, 13), (14, 13), (13, 14), (14, 14)]
        for position in roundabout:
            self.grid.properties["city_objects"].set_cell(position, 21)

    def step(self):
      print("Step ", self.steps)
      for semaphore in self.semaphores.values():
          semaphore.toggle_light()
          #print(f"Semaphore {semaphore.unique_id} has a state of: {semaphore.light_state}") #Debug
      self.agents.shuffle_do("step")
      print(self.grid.properties["city_objects"].data)


## Run


In [115]:
model = CityModel(1)
for _ in range(40):
    model.step()

Step  1
(9, 2) None
Current position: (9, 2)
Valid steps and their values:
--> All valid steps: 
Coordinate: (9, 1), Value: 0
Check steps:
Coordinate: (9, 1), Step_x: 9,  Step_y: 1
Old Position:  (9, 2)
New position: (9, 1)
[[ 0  0  0  0  0  0  0  0 19  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 19  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 20  2 20 20 18 18 20 20 20 20  0  0  0  0 20 12 20 20 20 20  0  0]
 [ 0  0 20 20 20 20  0  0 20 20 20 20  0  0  0  0 20 20 20 20 20 20  0  0]
 [ 0  0 20 20 20 20  0  0 20 20  9 20  0  0  0  0 20 20 20 20 20 20  0  0]
 [ 0  0 20 20 20 20  0  0 19  0  0  0  0  0  0  0 20 20 20 20 15 20  0  0]
 [ 0  0 20 20 20  6  0  0 19  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 20 20 20 20 18 18 20 20 20 20  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 20 20 20 20  0  0  7 20 20 20  0  0  0  0 20 20 20 20 16 20  0  0]
 [ 0 -1  0 20 20 20  0  0 20 20 20 20  0  0  0  0 20 20 20 20 20 20  0  0]
 [ 0  0 20 20 20 20  0  0 